In [1]:
import numpy as np
import pandas as pd
import lightgbm as lgb
import cv2
import torch
import pickle
from tqdm.auto import tqdm
import matplotlib.pyplot as plt

from sklearn.model_selection import ParameterGrid

In [2]:
import os
from PIL import Image
import imagehash
from tqdm.auto import tqdm
if 0:
    df1 = pd.read_csv('../input/petfinder-adoption-prediction/train/train.csv')
    df1['path'] = '../input/petfinder-adoption-prediction/train_images/' + df1['PetID'] + '-1.jpg'
    df1 = df1[df1['path'].map(os.path.exists)]

    df2 = pd.read_csv('../input/petfinder-pawpularity-score/train.csv')
    df2['path'] = df2['Id'].map(lambda x:str(f'../input/petfinder-pawpularity-score/train/{x}.jpg'))

    df1['hash'] = [imagehash.average_hash(Image.open(x))
                        for x in tqdm(df1['path'].values)]
    df2['hash'] = [imagehash.average_hash(Image.open(x))
                        for x in tqdm(df2['path'].values)]
    #train_df_prev['hash'] = [imagehash.average_hash(Image.open(x))
    #                    for x in tqdm(train_df_prev['path'].values)]

    df1.to_csv('df1.csv', index=False)
    df2.to_csv('df2.csv', index=False)
else:
    df1 = pd.read_csv('df1.csv')
    df2 = pd.read_csv('df2.csv')

In [3]:
df1.hash.nunique(), df1.shape

(14541, (14652, 26))

In [4]:
if 0:
    import tensorflow as tf
    from tensorflow.keras.models import load_model
    import tensorflow_similarity as tfsim
    tfsim.utils.tf_cap_memory() # Avoid GPU memory blow up
    IMG_SIZE = 300
    BATCH_SIZE = 64
    def load_and_preprocess_image(path):
        img = tf.io.read_file(path)
        img = tf.image.decode_jpeg(img, channels=3)
        img = tf.image.resize(img, [IMG_SIZE, IMG_SIZE])
        return img

    model = load_model('../../similarity/examples/sim.model')

    AUTOTUNE = tf.data.experimental.AUTOTUNE
    path_ds = tf.data.Dataset.from_tensor_slices(df1.path.values)
    ds = path_ds.map(load_and_preprocess_image, num_parallel_calls=AUTOTUNE)
    ds = ds.batch(BATCH_SIZE)
    ds = ds.prefetch(buffer_size=AUTOTUNE)
    embeds1 = model.predict(ds, batch_size=BATCH_SIZE)

    path_ds = tf.data.Dataset.from_tensor_slices(df2.path.values)
    ds = path_ds.map(load_and_preprocess_image, num_parallel_calls=AUTOTUNE)
    ds = ds.batch(BATCH_SIZE)
    ds = ds.prefetch(buffer_size=AUTOTUNE)
    embeds2 = model.predict(ds, batch_size=BATCH_SIZE)

    from cuml.neighbors import NearestNeighbors

    model = NearestNeighbors(n_neighbors=1, metric="cosine")
    model.fit(embeds1)
    distances, indices = model.kneighbors(embeds2)

    df2['distance'] = distances
    df2['hash'] = indices
    df2.loc[df2['distance'] < 1.0e-4, 'hash'] = -1
    df1['hash'] = df1.index.values

In [5]:
df1.shape, df1.hash.nunique()

((14652, 26), 14541)

In [6]:
if 0:
    list_label = []
    list_det = []
    for path in tqdm(df2['path'].values):
        #result = inference_detector(model, path)
        #list_det.append(get_det_result(result))
        image = cv2.imread(path)
        list_label.append(image.shape)

        #image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) / 255

        #image = cv2.resize(image, (240, 240))
        #image = np.moveaxis(image, -1, 0)
        #image = torch.tensor([image], dtype=torch.float32, device=device)
        #list_label.append(model(image).cpu().numpy()[0].argmax())

    #df['resnet_label'] = list_label
    list_label = np.array(list_label)
    df2['img_h'] = list_label[:, 0]
    df2['img_w'] = list_label[:, 1]



  0%|          | 0/9912 [00:00<?, ?it/s]

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
model = TfidfVectorizer(stop_words=None,#np.load('stop_words.npy').tolist(),
                            binary=False,
                            #norm='l1',
                            #analyzer='char_wb',
                            #ngram_range=(2, 5),
                            # tokenizer=tokenizer,
                            # max_features=50000,
                            #max_df=50,
                            min_df=2,
                            dtype=np.float32)

#data = model.fit_transform(df1['Description'].fillna('').str.lower().values)
#df1['desc_sum'] = np.array(data.sum(axis=1))[:, 0]
#df1['desc_max'] = np.array(data.max(axis=1).todense())[:, 0]
#df1['desc_mean'] = df1['desc_sum'] / (data.getnnz(axis=1))


data = model.fit_transform(df1['Name'].fillna('').str.lower().values)
print(data.shape)
df1['name_sum'] = np.array(data.sum(axis=1))[:, 0]
df1['name_max'] = np.array(data.max(axis=1).todense())[:, 0]
df1['name_mean'] =  df1['name_sum'] / data.getnnz(axis=1)

(14652, 2175)


In [8]:
#vocab = ['abandoned', 'active', 'adopt', 'adopted', 'adoption', 'after', 'all', 'an', 'and', 'are', 'as', 'at', 'be', 'been', 'being', 'bring', 'but', 'by', 'call', 'can', 'care', 'cat', 'contact', 'cute', 'dog', 'family', 'for', 'forever', 'found', 'from', 'give', 'good', 'has', 'have', 'he', 'healthy', 'heavy', 'her', 'him', 'home', 'if', 'in', 'interested', 'is', 'it', 'likes', 'looking', 'loving', 'male', 'me', 'months', 'mother', 'must', 'my', 'new', 'no', 'not', 'of', 'old', 'on', 'or', 'other', 'out', 'playful', 'please', 'puppy', 'rescued', 'she', 'so', 'spayed', 'take', 'than', 'that', 'the', 'they', 'this', 'to', 'trained', 'vaccination', 'very', 'visit', 'was', 'we', 'well', 'whatsapp', 'who', 'will', 'with', 'would', 'you']
vocab = ['active', 'adopt', 'adoption', 'all', 'an', 'and', 'are', 'as', 'at', 'be', 'bring', 'but', 'by', 'call', 'can', 'care', 'cat', 'contact', 'cute', 'dog', 'family', 'for', 'from', 'give', 'good', 'has', 'have', 'he', 'healthy', 'her', 'him', 'home', 'if', 'in', 'interested', 'is', 'it', 'looking', 'loving', 'me', 'must', 'my', 'no', 'not', 'of', 'old', 'on', 'or', 'playful', 'please', 'rescued', 'she', 'spayed', 'that', 'the', 'they', 'this', 'to', 'trained', 'very', 'was', 'we', 'whatsapp', 'will', 'with', 'you']
#vocab = ['active', 'adopt', 'adoption', 'all', 'an', 'and', 'are', 'as', 'at', 'be', 'bring', 'but', 'by', 'can', 'care', 'cat', 'contact', 'cute', 'dog', 'family', 'for', 'give', 'good', 'has', 'have', 'he', 'healthy', 'her', 'him', 'home', 'if', 'in', 'interested', 'is', 'it', 'looking', 'loving', 'me', 'must', 'my', 'no', 'not', 'of', 'old', 'on', 'or', 'playful', 'please', 'rescued', 'she', 'spayed', 'that', 'the', 'they', 'this', 'to', 'trained', 'very', 'was', 'we', 'will', 'with', 'you']
model = TfidfVectorizer(stop_words=None,#np.load('stop_words.npy').tolist(),
                            binary=False,
                            #norm='l1',
                            #analyzer='char_wb',
                            #ngram_range=(2, 5),
                            # tokenizer=tokenizer,
                            # max_features=50000,
                            #max_df=50,
                            #vocabulary=vocab,
                            min_df=2,
                            dtype=np.float32)

tmp = df1['Description'].fillna('').str.lower()# + ' ' + df1['Name'].fillna('').str.lower() 
#data = model.fit_transform(df1['Description'].fillna('').str.lower().values)
data = model.fit_transform(tmp.values)

map_i2w = {v:k for k, v in model.vocabulary_.items()}
#data = np.array(data.todense())

if 1:
    for w in vocab:
        if w in model.vocabulary_:
            df1[f'tfidf_{w}'] = np.array(data[:, model.vocabulary_[w]].todense())
else:
    for w, i in model.vocabulary_.items():
        df1[f'tfidf_{w}'] = np.array(data[:, i].todense())
    
#for w in ['white', 'boy']:
#    df1[f'tfidf_{w}'] = np.array(data[:, model.vocabulary_[w]].todense())



In [9]:
#df1.to_csv('train_last_comp.csv', index=False)
df1['hash'] = df1['hash'].astype(str)
df2['hash'] = df2['hash'].astype(str)
df = pd.merge(df2, df1, how='left', on='hash')
df.shape

(9939, 112)

In [10]:
df['len_desc'] = df['Description'].fillna('').map(len)
df['len_word_desc'] = df['Description'].fillna('').map(lambda x: len(x.split()))
for n in ['No Name Yet']:#, 'No Name', 'Unknown']:
    df['Name'] = df['Name'].fillna('').replace(n, '')
df['len_name'] = df['Name'].map(len)
df['len_word_name'] = df['Name'].fillna('').map(lambda x: len(x.split()))

df['cnt_RescuerID'] = df['RescuerID'].map(df1['RescuerID'].value_counts())

In [11]:
#df = pd.read_csv('../exps/exp1_swin_large_patch4_window12_384/train_cv_score.csv')
#df = pd.read_csv('ens_exp15.csv')

In [12]:
df['pred'] = df['path_x'].map(pd.read_csv('ens_exp15.csv', index_col='path')['pred'])
df['pred'] += df['path_x'].map(pd.read_csv('ens_exp9.csv', index_col='path')['pred'])
df['pred'] /= 2
df['pred']

0       0.471986
1       0.402806
2       0.440848
3       0.603114
4       0.386127
          ...   
9934    0.276084
9935    0.371682
9936    0.293679
9937    0.469386
9938    0.434194
Name: pred, Length: 9939, dtype: float64

In [13]:
if 0:
    from transformers import pipeline, AutoTokenizer
    model_name = "bert-base-multilingual-uncased"
    #tokenizer = AutoTokenizer.from_pretrained(model_name, max_length=512)
    feature_extraction = pipeline('feature-extraction', model=model_name, tokenizer=model_name, padding=True, truncation=True, device=0)
    features = feature_extraction(df['Name'].fillna('').str.lower().values.tolist())
    #features = feature_extraction(df['Description'].fillna('').str.lower().values.tolist())

    data = np.stack([np.mean(f[0], axis=0) for f in features])
    for i in range(data.shape[1]):
        df[f'bert_{i}'] = data[:, i]

In [14]:
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold

df['fold'] = -1


N_FOLDS = 5
strat_kfold = StratifiedKFold(n_splits=5, random_state=365, shuffle=True)
for i, (_, train_index) in enumerate(strat_kfold.split(df.index, df['Pawpularity'])):
    df.loc[train_index, 'fold'] = i

/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_split.py:672: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)


In [31]:
COL_FEATURES = ['Blur', 
                #'Face', 
                #'Occlusion',
                'Eyes',
                #'Accessory',
               #'Group',
                #'Human',
                #'Near',
                #'Collage',
                #'Info',# 
                #'Action',
                
                'pred',
                
                'len_desc', 'len_name',
                #'breed_label',# 'breed_prob',
                #'len_word_desc', 'len_word_name',
                #'isin'
                #'breed_label', 'breed_prob',
                'img_h', 'img_w',
                #'det_label', 'det_left', 'det_top', 'det_right', 'det_bottom', 'det_prob'
               ] + df1.drop(['Name', 'RescuerID', 'Description', 'PetID', 'path', 'hash',
                             'Type', 'VideoAmt', 'Sterilized', 'MaturitySize', 'Health', 'Dewormed',# 'Breed1'
                            ], axis=1).columns.tolist() + [c for c in df.columns if 'bert_' in c]

#COL_FEATURES = COL_FEATURES[:7]

In [32]:
if df['Pawpularity'].values.min() == 0:
    df['Pawpularity'] += 1

In [33]:
def rmse(preds, train_data):
    labels = train_data.get_label()
    #import pdb;pdb.set_trace()
    loss = np.sqrt(((labels - preds.clip(0.01, 1)) ** 2).mean()) * 100
    
    #loss = np.sqrt(((labels - preds.reshape(-1, 100).argmax(axis=1)) ** 2).mean())
    return 'rmse', loss, False


def train(fold, param):
     
    X_train = df.loc[df['fold'] != fold, COL_FEATURES]
    y_train = df.loc[df['fold'] != fold, 'Pawpularity'].values  / 100
    
    X_valid = df.loc[df['fold'] == fold, COL_FEATURES]
    y_valid = df.loc[df['fold'] == fold, 'Pawpularity'].values  / 100
    
    train_data = lgb.Dataset(X_train, label=y_train)
    valid_data = lgb.Dataset(X_valid, label=y_valid)
    
    model = lgb.train(param,
                      train_data,
                      valid_sets=valid_data,
                      #early_stopping_rounds=50,
                      verbose_eval=1000,
                      feval=rmse
                      )
    return model

def train_all(param):
     
    X_train = df.loc[:, COL_FEATURES]
    y_train = df.loc[:, 'Pawpularity'].values  / 100
    
    X_valid = df.loc[:, COL_FEATURES]
    y_valid = df.loc[:, 'Pawpularity'].values  / 100
    
    train_data = lgb.Dataset(X_train, label=y_train)
    valid_data = lgb.Dataset(X_valid, label=y_valid)
    
    model = lgb.train(param,
                      train_data,
                      valid_sets=valid_data,
                      early_stopping_rounds=50,
                      verbose_eval=1000,
                      feval=rmse
                      )
    return model

In [34]:
all_params = {'objective': ['tweedie'],
              'tweedie_variance_power': [1.2],
             'verbosity': [-1],
             'boosting_type': ['gbdt'],
             'feature_pre_filter': [False],
             #'bagging_fraction': [0.7, 0.8, 0.9, 1],
             'bagging_freq': [1],
             'num_iterations': [10000],
             'early_stopping_round': [100],
             'n_jobs': [16],
             'seed': [114],
             'metric':  ['None'],  # trial.suggest_categorical('metric', ['auc', 'binary_logloss', ]), #'auc',
             'learning_rate': [0.05],
              'lambda_l1': [0],
              'lambda_l2': [1],
              'min_child_samples': [170],
              'num_leaves': [7],
              'feature_fraction': [0.8],
              'min_gain_to_split': [0.02],
              'linear_tree': [False],
              #'max_bins': [8, 16, 32, 62, 128, 256, 512]
             }

In [35]:
best_score = 1.0e10
best_param = None
for param in tqdm(ParameterGrid(all_params)):
    print(param)
    list_loss = []
    list_imp = []
    list_num = []
    for fold in range(5):
        model = train(fold, param)
        sc = model.best_score['valid_0']['rmse']# * 100
        list_loss.append(sc)
        list_num.append(model.best_iteration)
        
        imp = pd.DataFrame(model.feature_importance(importance_type='gain'), columns=['imp'])
        imp['col'] = COL_FEATURES
        list_imp.append(imp.set_index('col'))
    sc = np.mean(list_loss)
    if sc < best_score:
        best_score = sc
        best_param = param

  0%|          | 0/1 [00:00<?, ?it/s]

{'bagging_freq': 1, 'boosting_type': 'gbdt', 'early_stopping_round': 100, 'feature_fraction': 0.8, 'feature_pre_filter': False, 'lambda_l1': 0, 'lambda_l2': 1, 'learning_rate': 0.05, 'linear_tree': False, 'metric': 'None', 'min_child_samples': 170, 'min_gain_to_split': 0.02, 'n_jobs': 16, 'num_iterations': 10000, 'num_leaves': 7, 'objective': 'tweedie', 'seed': 114, 'tweedie_variance_power': 1.2, 'verbosity': -1}
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[96]	valid_0's rmse: 17.1691


/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[290]	valid_0's rmse: 17.1653
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[145]	valid_0's rmse: 17.3397
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[197]	valid_0's rmse: 17.1704
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[185]	valid_0's rmse: 16.7856


In [36]:
17.122953793907463
17.12237161917507
17.12206798235878
17.118772493988214
17.11253518980743
17.10951939364919
17.10667371043545
print(best_param)
print(best_score)

{'bagging_freq': 1, 'boosting_type': 'gbdt', 'early_stopping_round': 100, 'feature_fraction': 0.8, 'feature_pre_filter': False, 'lambda_l1': 0, 'lambda_l2': 1, 'learning_rate': 0.05, 'linear_tree': False, 'metric': 'None', 'min_child_samples': 170, 'min_gain_to_split': 0.02, 'n_jobs': 16, 'num_iterations': 10000, 'num_leaves': 7, 'objective': 'tweedie', 'seed': 114, 'tweedie_variance_power': 1.2, 'verbosity': -1}
17.126018982016205


In [37]:
best_param['num_iterations'] = round(np.mean(list_num) * 1.1)

In [38]:
model = train_all(best_param)

Training until validation scores don't improve for 100 rounds


/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "


Did not meet early stopping. Best iteration is:
[201]	valid_0's rmse: 16.6721


In [39]:
aaa = 0
for i in range(5):
    aaa += np.sqrt(((df.loc[df['fold']==i, 'pred'] * 100 - df.loc[df['fold']==i, 'Pawpularity']) ** 2).mean())
aaa / 5

17.275769284357725

In [40]:
with open('models/model_lgb_last_comp_all.pkl', 'wb') as f:
    pickle.dump(model, f, -1)

In [41]:
#imp = pd.DataFrame(model.feature_importance(importance_type='gain'), columns=['imp'])
#imp['col'] = COL_FEATURES
pd.options.display.max_rows = 3000
imp = sum(list_imp) / 5
#imp = imp[imp.imp > 0]
imp.sort_values('imp', ascending=False)

,imp
col,
pred,3088.936279
Breed1,56.815157
Age,15.379413
img_w,15.110815
PhotoAmt,14.319040
tfidf_is,14.245327
AdoptionSpeed,12.965359
img_h,12.604309
name_max,12.478088


In [42]:
print([c[6:] for c in imp.index if 'tfidf_' in c])

['active', 'adopt', 'adoption', 'all', 'an', 'and', 'are', 'as', 'at', 'be', 'bring', 'but', 'by', 'call', 'can', 'care', 'cat', 'contact', 'cute', 'dog', 'family', 'for', 'from', 'give', 'good', 'has', 'have', 'he', 'healthy', 'her', 'him', 'home', 'if', 'in', 'interested', 'is', 'it', 'looking', 'loving', 'me', 'must', 'my', 'no', 'not', 'of', 'old', 'on', 'or', 'playful', 'please', 'rescued', 'she', 'spayed', 'that', 'the', 'they', 'this', 'to', 'trained', 'very', 'was', 'we', 'whatsapp', 'will', 'with', 'you']


In [43]:
print([c for c in imp.index if 'tfidf_name_' in c])

[]


In [44]:
df.len_desc

0       102
1       601
2        88
3         0
4         0
       ... 
9934      0
9935      0
9936      0
9937    157
9938      0
Name: len_desc, Length: 9939, dtype: int64

In [45]:
model.feature_name()

['Blur',
 'Eyes',
 'pred',
 'len_desc',
 'len_name',
 'img_h',
 'img_w',
 'Age',
 'Breed1',
 'Breed2',
 'Gender',
 'Color1',
 'Color2',
 'Color3',
 'FurLength',
 'Vaccinated',
 'Quantity',
 'Fee',
 'State',
 'PhotoAmt',
 'AdoptionSpeed',
 'name_sum',
 'name_max',
 'name_mean',
 'tfidf_active',
 'tfidf_adopt',
 'tfidf_adoption',
 'tfidf_all',
 'tfidf_an',
 'tfidf_and',
 'tfidf_are',
 'tfidf_as',
 'tfidf_at',
 'tfidf_be',
 'tfidf_bring',
 'tfidf_but',
 'tfidf_by',
 'tfidf_call',
 'tfidf_can',
 'tfidf_care',
 'tfidf_cat',
 'tfidf_contact',
 'tfidf_cute',
 'tfidf_dog',
 'tfidf_family',
 'tfidf_for',
 'tfidf_from',
 'tfidf_give',
 'tfidf_good',
 'tfidf_has',
 'tfidf_have',
 'tfidf_he',
 'tfidf_healthy',
 'tfidf_her',
 'tfidf_him',
 'tfidf_home',
 'tfidf_if',
 'tfidf_in',
 'tfidf_interested',
 'tfidf_is',
 'tfidf_it',
 'tfidf_looking',
 'tfidf_loving',
 'tfidf_me',
 'tfidf_must',
 'tfidf_my',
 'tfidf_no',
 'tfidf_not',
 'tfidf_of',
 'tfidf_old',
 'tfidf_on',
 'tfidf_or',
 'tfidf_playful',
 '